In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pytorch-metric-learning


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.1/119.1 kB 1.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [3]:
import pandas as pd
import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from pytorch_metric_learning import distances, losses, miners, reducers, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator

folder = '/content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES'
all_pictures = os.listdir(folder)

file_path = "/content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES.csv"
df = pd.read_csv(file_path)

# Reemplazar el prefijo "c." con nada en la columna 'date'
#df['date'] = df['date'].str.replace('c.', '').astype('int', errors='ignore')


print(df.head())

NUM_CLASSES = len(df['date'].unique().tolist())

print("Number of classes:", NUM_CLASSES)


                  artist  date      genre  pixelsx  pixelsy  size_bytes  \
0  Ernst Ludwig Kirchner  1913  landscape   4908.0   3684.0  10682834.0   
1          Koloman Moser  1913  landscape   4716.0   3420.0  10933062.0   
2          Koloman Moser  1913  landscape   4722.0   3102.0   9018058.0   
3       Ferdinand Hodler  1913   portrait   3156.0   4638.0   8453089.0   
4          Koloman Moser  1913     design   4734.0   2622.0   8317658.0   

    source                 style  \
0  wikiart         Expressionism   
1  wikiart             Symbolism   
2  wikiart             Symbolism   
3  wikiart  Art Nouveau (Modern)   
4  wikiart  Art Nouveau (Modern)   

                                            title    artist_group  in_train  \
0                           Red Tree on the Beach  train_and_test      True   
1                                 Mountain ranges  train_and_test      True   
2                          Landscape in Semmering  train_and_test      True   
3      Valentine

In [4]:
import os
import shutil
import random
import pandas as pd


In [5]:
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

In [6]:
# GROUP THE FILES BY EACH date
import random
def group_files_per_date():
  data_per_date = {}

  # Iterate over each row of the DataFrame
  for index, row in df.iterrows():
      date = row['date']
      new_filename = row['new_filename']

      # Check if the date is already in the dictionary
      if date in data_per_date:
          # Add the new_filename to the existing list of the date
          data_per_date[date].append(new_filename)
      else:
          # Create a new list for the date and add the new_filename
          data_per_date[date] = [new_filename]

  return data_per_date

# SPLIT THE FILES INTO TRAIN AND TEST SETS (CREATE NEW DATASETS)
def split_test_train(data_per_date, train_percentage = 0.8):

  TRAIN_names = []
  TEST_names = []

  # Iterate over each date and their 'new_filename'
  for date, filenames in data_per_date.items():
      # Calculate the number of files for training and testing
      total_files = len(filenames)
      num_train = int(train_percentage * total_files)

      # Shuffle the filenames to avoid selection biases
      random.shuffle(filenames)

      # Divide the filenames into train and test
      train_filenames = filenames[:num_train]
      test_filenames = filenames[num_train:]

      # Store the divided filenames into train and test
      TRAIN_names.extend(train_filenames)
      TEST_names.extend(test_filenames)
  return TRAIN_names, TEST_names



# CREATE TEST and TRAIN folders
def create_train_test_folders(folder, TRAIN_names, TEST_names):
    # Output folder for TRAIN and TEST
    train_folder = os.path.join(folder, 'TRAIN')
    test_folder = os.path.join(folder, 'TEST')

    # Create TRAIN and TEST folders if they don't exist
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(test_folder, exist_ok=True)

    # Move images from TRAIN to TRAIN folder
    for filename in TRAIN_names:
        src = os.path.join(folder, filename)
        dst = os.path.join(train_folder, filename)
        shutil.move(src, dst)

    # Move images from TEST to TEST folder
    for filename in TEST_names:
        src = os.path.join(folder, filename)
        dst = os.path.join(test_folder, filename)
        shutil.move(src, dst)

    # Create df_train with rows whose 'new_filename' is in TRAIN_names and keep only 'date' and 'new_filename' columns
    train_dataframe = df[df['new_filename'].isin(TRAIN_names)][['date', 'new_filename']]
    # File path for the CSV file
    file_path = os.path.join(folder, 'train_dataframe.csv')
    # Save the DataFrame as CSV
    train_dataframe.to_csv(file_path, index=False)

    # Create df_test with rows whose 'new_filename' is in TEST_names and keep only 'date' and 'new_filename' columns
    test_dataframe = df[df['new_filename'].isin(TEST_names)][['date', 'new_filename']]
    # File path for the CSV file
    file_path = os.path.join(folder, 'test_dataframe.csv')
    # Save the DataFrame as CSV
    test_dataframe.to_csv(file_path, index=False)

    return train_dataframe, test_dataframe

if not (os.path.exists(os.path.join(folder, 'TRAIN')) and os.path.exists(os.path.join(folder, 'TEST')) and os.path.exists(os.path.join(folder, 'train_dataframe.csv'))):
  # Group files per date
  data_per_date = group_files_per_date()
  # Print the number of files for each date
  print("- date and numer of images:")
  for key in data_per_date.keys():
    print(key, len(data_per_date[key]))

  # Split data
  TRAIN_names, TEST_names = split_test_train(data_per_date)

  # Use the function to create TRAIN and TEST folders
  train_dataframe, test_dataframe = create_train_test_folders(folder, TRAIN_names, TEST_names)
  print("\nImages successfully moved to TRAIN and TEST folders.")
else:
  print("TRAIN and TEST folders already exist. Reading train and test DataFrames from CSV files...")

  # Read train and test DataFrames from CSV files
  train_folder = os.path.join(folder, 'TRAIN')
  test_folder = os.path.join(folder, 'TEST')

  train_images = set(os.listdir(train_folder))
  test_images = set(os.listdir(test_folder))

  train_dataframe = df[df['new_filename'].isin(train_images)].copy()
  test_dataframe = df[df['new_filename'].isin(test_images)].copy()

  train_dataframe = train_dataframe[['new_filename', 'date']]
  test_dataframe = test_dataframe[['new_filename', 'date']]

  print("Train DataFrame:", train_dataframe.shape)
  print("Test DataFrame:", test_dataframe.shape)


TRAIN and TEST folders already exist. Reading train and test DataFrames from CSV files...
Train DataFrame: (1200, 2)
Test DataFrame: (300, 2)


In [7]:
train_dataframe.groupby('date').size().reset_index(name='count')

,date,count
0,1885,80
1,1888,80
2,1889,80
3,1890,80
4,1895,80
5,1907,80
6,1908,80
7,1909,80
8,1910,80
9,1911,80


In [8]:
test_dataframe.groupby('date').size().reset_index(name='count')

,date,count
0,1885,20
1,1888,20
2,1889,20
3,1890,20
4,1895,20
5,1907,20
6,1908,20
7,1909,20
8,1910,20
9,1911,20


In [9]:
'''
import os
from PIL import Image
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from pytorch_metric_learning import distances, losses, miners, reducers, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator

# Define the transformation to convert images to PyTorch tensors
transf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4902860351171657, 0.4327027123349515, 0.3679902021355657], std=[0.22134062718873712, 0.20430693703606892, 0.18851782142531143])  # Normalization for color images
])
# Paths of the training and testing folders
train_folder = '/content/drive/My Drive/PROJECTE DEEP LEARNING/TOP5_ARTISTS_WITH_100_PICTURES/TRAIN'
test_folder = '/content/drive/My Drive/PROJECTE DEEP LEARNING/TOP5_ARTISTS_WITH_100_PICTURES/TEST'

# Get the file names of training and testing images from the dataframes
train_filenames = train_dataframe['new_filename'].tolist()
test_filenames = test_dataframe['new_filename'].tolist()
# Create empty lists for images and labels
train_images = []
train_labels = []
test_images = []
test_labels = []
#if not (os.path.exists(os.path.join(folder, 'TRAIN')) and os.path.exists(os.path.join(folder, 'TEST'))):

  # Load training images
for filename in train_filenames:
    img_path = os.path.join(train_folder, filename)
    img = Image.open(img_path)
    img_tensor = transf(img)
    train_images.append(img_tensor)
    # Get label from the dataframe
    label = train_dataframe[train_dataframe['new_filename'] == filename]['date'].iloc[0]
    train_labels.append(label)

# Load testing images
for filename in test_filenames:
    img_path = os.path.join(test_folder, filename)
    img = Image.open(img_path)
    img_tensor = transf(img)
    test_images.append(img_tensor)
    # Get label from the dataframe
    label = test_dataframe[test_dataframe['new_filename'] == filename]['date'].iloc[0]
    test_labels.append(label)

# Convert labels to tensor
train_labels_tensor = torch.tensor(train_labels, dtype=torch.float32)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.float32)

# Create PyTorch datasets
train_dataset = TensorDataset(torch.stack(train_images), train_labels_tensor)
test_dataset = TensorDataset(torch.stack(test_images), test_labels_tensor)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=50, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=50, shuffle=False)
'''

"\nimport os\nfrom PIL import Image\nimport torch\nfrom torchvision import transforms\nfrom torch.utils.data import DataLoader, TensorDataset\nimport pandas as pd\nimport torch.nn as nn\nimport torch.optim as optim\nimport torch.nn.functional as F\nfrom pytorch_metric_learning import distances, losses, miners, reducers, testers\nfrom pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator\n\n# Define the transformation to convert images to PyTorch tensors\ntransf = transforms.Compose([\n    transforms.ToTensor(),\n    transforms.Normalize(mean=[0.4902860351171657, 0.4327027123349515, 0.3679902021355657], std=[0.22134062718873712, 0.20430693703606892, 0.18851782142531143])  # Normalization for color images\n])\n# Paths of the training and testing folders\ntrain_folder = '/content/drive/My Drive/PROJECTE DEEP LEARNING/TOP5_ARTISTS_WITH_100_PICTURES/TRAIN'\ntest_folder = '/content/drive/My Drive/PROJECTE DEEP LEARNING/TOP5_ARTISTS_WITH_100_PICTURES/TEST'\n\n# Get the f

In [10]:
'''
import matplotlib.pyplot as plt
import numpy as np

def show_images(dataset, labels, class_name, num_images=4):
    # Find indices of images belonging to the specified class
    class_indices = [idx for idx, label in enumerate(labels) if label == class_name]
    # Randomly select 4 indices
    selected_indices = np.random.choice(class_indices, num_images)

    # Plot the images
    fig, axes = plt.subplots(1, num_images, figsize=(15, 3))
    fig.suptitle(f"Class {class_name} Images")
    for i, idx in enumerate(selected_indices):
        img, label = dataset[idx]
        img = img.permute(1, 2, 0)  # Convert from (C, H, W) to (H, W, C) for visualization
        axes[i].imshow(img)
        axes[i].axis('off')
    plt.show()

# Example: Show 4 images of class 0 from the training set
show_images(train_dataset, train_labels_tensor, class_name=1913)

# Example: Show 4 images of class 1 from the testing set
show_images(test_dataset, test_labels_tensor, class_name=1913)
'''

'\nimport matplotlib.pyplot as plt\nimport numpy as np\n\ndef show_images(dataset, labels, class_name, num_images=4):\n    # Find indices of images belonging to the specified class\n    class_indices = [idx for idx, label in enumerate(labels) if label == class_name]\n    # Randomly select 4 indices\n    selected_indices = np.random.choice(class_indices, num_images)\n\n    # Plot the images\n    fig, axes = plt.subplots(1, num_images, figsize=(15, 3))\n    fig.suptitle(f"Class {class_name} Images")\n    for i, idx in enumerate(selected_indices):\n        img, label = dataset[idx]\n        img = img.permute(1, 2, 0)  # Convert from (C, H, W) to (H, W, C) for visualization\n        axes[i].imshow(img)\n        axes[i].axis(\'off\')\n    plt.show()\n\n# Example: Show 4 images of class 0 from the training set\nshow_images(train_dataset, train_labels_tensor, class_name=1913)\n\n# Example: Show 4 images of class 1 from the testing set\nshow_images(test_dataset, test_labels_tensor, class_name=

In [11]:
import os
from PIL import Image
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from pytorch_metric_learning import distances, losses, miners, reducers, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator



# NETWORK

In [12]:
from pytorch_metric_learning import losses

In [13]:
### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ###
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(64 * 110 * 110, 128)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        return x

In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ###
from torchvision import datasets, transforms

from pytorch_metric_learning import distances, losses, miners, reducers, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator


### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ###
def train(model, loss_func, mining_func, device, train_loader, optimizer, epoch):
    model.train()
    epoch_loss = 0.0  # Inicializa la pérdida acumulada en la época

    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        embeddings = model(data)
        indices_tuple = mining_func(embeddings, labels)
        loss = loss_func(embeddings, labels, indices_tuple)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()  # Acumula la pérdida del lote en la pérdida de la época

    epoch_loss /= len(train_loader)  # Calcula el promedio de la pérdida de la época
    print("Epoch {}: Average Loss = {}".format(epoch, epoch_loss))  # Imprime la pérdida promedio de la época

    return epoch_loss  # Devuelve la pérdida promedio de la época

### convenient function from pytorch-metric-learning ###
def get_all_embeddings(dataset, model):
    tester = testers.BaseTester()
    return tester.get_all_embeddings(dataset, model)


# LA PRIMERA FUNCIO DE test CALCULA ACC AMB % D'ACIERTO, LA SEGONA ES LA K ESTAVA JA DEL INICIO (VERSION AKSEL)

def test(model, test_loader, device): # Versio %
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            print("PREDICTED", predicted)
            print(labels.size(0))
            print("REAL:", labels)
            print(predicted.shape)
    accuracy = 100 * correct / total
    print('Accuracy of the network on the test images: %d %%' % accuracy)
    return accuracy
'''

def test(train_set, test_set, model, accuracy_calculator, loss_func,): # Versio NO %
    train_embeddings, train_labels = get_all_embeddings(train_set, model)
    test_embeddings, test_labels = get_all_embeddings(test_set, model)
    train_labels = train_labels.squeeze(1)
    test_labels = test_labels.squeeze(1)
    print("Computing accuracy")
    accuracies = accuracy_calculator.get_accuracy(
        test_embeddings, test_labels, train_embeddings, train_labels, False
    )
    print("Test set accuracy (Precision@1) = {}".format(accuracies["precision_at_1"]))
    return accuracies["precision_at_1"]
'''

'\n\ndef test(train_set, test_set, model, accuracy_calculator, loss_func,): # Versio NO %\n    train_embeddings, train_labels = get_all_embeddings(train_set, model)\n    test_embeddings, test_labels = get_all_embeddings(test_set, model)\n    train_labels = train_labels.squeeze(1)\n    test_labels = test_labels.squeeze(1)\n    print("Computing accuracy")\n    accuracies = accuracy_calculator.get_accuracy(\n        test_embeddings, test_labels, train_embeddings, train_labels, False\n    )\n    print("Test set accuracy (Precision@1) = {}".format(accuracies["precision_at_1"]))\n    return accuracies["precision_at_1"]\n'

In [15]:
# Paths of the training and testing folders
train_folder = '/content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TRAIN'
test_folder = '/content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TEST'

In [16]:
device = torch.device("cuda")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4902860351171657, 0.4327027123349515, 0.3679902021355657], std=[0.22134062718873712, 0.20430693703606892, 0.18851782142531143])  # Normalization for color images
])

In [17]:
batch_size = 50

In [18]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import os
from PIL import Image


# Define la clase Dataset personalizada para cargar tus datos
class CustomDataset(Dataset):
    def __init__(self, dataframe, folder, transform=None):
        self.dataframe = dataframe
        self.folder = folder
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.folder, self.dataframe.iloc[idx, 0])
        image = Image.open(img_name)
        date = self.dataframe.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        return image, date


# Definir las rutas de las carpetas de entrenamiento y prueba
train_folder = '/content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TRAIN'
test_folder = '/content/drive/My Drive/PROJECTE DEEP LEARNING/TOP15_DATAS_WITH_100_PICTURES/TEST'

# Crear instancias del conjunto de datos personalizado
train_dataset = CustomDataset(train_dataframe, train_folder, transform=transform)
test_dataset = CustomDataset(test_dataframe, test_folder, transform=transform)

# Crear los DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [19]:
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)


In [20]:
### pytorch-metric-learning stuff ###
distance = distances.CosineSimilarity()
reducer = reducers.ThresholdReducer(low=0)
loss_func = losses.TripletMarginLoss(margin=0.2, distance=distance, reducer=reducer)
mining_func = miners.TripletMarginMiner(
    margin=0.2, distance=distance, type_of_triplets="semihard"
)

## Les 2 celas comentades shan de fer servir si es vol utilitzar la versio de la funcio "test" que NO fa el %. Primer, s'ha de fer els 2 unistall y despres els 2 install i reiniciar entorno ejecucion (ho demanara sol).


In [21]:
'''
#!pip uninstall faiss-gpu
#!pip uninstall faiss-cpu
!pip install faiss-gpu
!pip install faiss-cpu
'''

'\n#!pip uninstall faiss-gpu\n#!pip uninstall faiss-cpu\n!pip install faiss-gpu\n!pip install faiss-cpu\n'

In [22]:
'''
import faiss


accuracy_calculator = AccuracyCalculator(include=("precision_at_1",), k=1)
### pytorch-metric-learning stuff ###
'''

'\nimport faiss\n\n\naccuracy_calculator = AccuracyCalculator(include=("precision_at_1",), k=1)\n### pytorch-metric-learning stuff ###\n'

In [34]:
num_epochs = 30
for epoch in range(num_epochs):
    train(model, loss_func, mining_func, device, train_loader, optimizer, epoch)
    # test(model, test_loader, device)

Epoch 0: Average Loss = 0.10374512026707332
Epoch 1: Average Loss = 0.10394578116635482
Epoch 2: Average Loss = 0.10387686143318813
Epoch 3: Average Loss = 0.10432885990788539
Epoch 4: Average Loss = 0.10498233884572983
Epoch 5: Average Loss = 0.10355781173954408
Epoch 6: Average Loss = 0.10353940135488908
Epoch 7: Average Loss = 0.10144900561620791
Epoch 8: Average Loss = 0.10338809651633103
Epoch 9: Average Loss = 0.10310137209792931
Epoch 10: Average Loss = 0.10452382204433282
Epoch 11: Average Loss = 0.10366725611190002
Epoch 12: Average Loss = 0.10262537406136592
Epoch 13: Average Loss = 0.10261200244228046
Epoch 14: Average Loss = 0.1027551805600524
Epoch 15: Average Loss = 0.10201624160011609
Epoch 16: Average Loss = 0.10269618468979995
Epoch 17: Average Loss = 0.10278139698008697
Epoch 18: Average Loss = 0.10286446226139863
Epoch 19: Average Loss = 0.10173067512611549
Epoch 20: Average Loss = 0.1031162915751338
Epoch 21: Average Loss = 0.10263247415423393
Epoch 22: Average Loss

## Versio per evaluar, a partir dels embedings de train es construeix un classificador de Knn i es calcula l'acc (no shauria de fer aixi)

In [35]:
def evaluate(model, loader, device):
    model.eval()
    embeddings = []
    labels = []
    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            embeddings.append(output.cpu().numpy())
            labels.append(target.cpu().numpy())
    embeddings = np.vstack(embeddings)
    labels = np.hstack(labels)
    return embeddings, labels

In [37]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np


# Get embeddings and labels for train and test sets
train_embeddings, train_labels = evaluate(model, train_loader, device)
test_embeddings, test_labels = evaluate(model, test_loader, device)

# Use k-NN to classify test embeddings based on train embeddings
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(train_embeddings, train_labels)
predicted_labels = knn.predict(test_embeddings)

# Compute accuracy
accuracy = np.mean(predicted_labels == test_labels)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 9.33%
